In [2]:
import os
import re
import pickle
import pandas as pd
from bs4 import BeautifulSoup as bs

In [3]:
pattern = re.compile(r'i=([A-Z]+)&t=([0-9]+)&s=([A-Z]+)')
paths = sorted([
    os.path.join(path, file)
    for path, dirs, files in os.walk('.')
    for file in files
    if pattern.match(file) is not None])

# display(paths)

In [4]:
query_dicts = []
for path in paths:
    file = os.path.basename(path)
    match = pattern.match(file)
    if match is not None:
        query_dicts.append({
            'path': path,
            'institute_id': match[1],
            'term_id': match[2],
            'school_id': match[3]})
        
df_query = pd.DataFrame(query_dicts).astype({'term_id': 'int64'})
display(df_query)

,path,institute_id,term_id,school_id
0,./2023-06-06/i=HAW&t=202130&s=ABRP,HAW,202130,ABRP
1,./2023-06-06/i=HAW&t=202130&s=ACC,HAW,202130,ACC
2,./2023-06-06/i=HAW&t=202130&s=AEC,HAW,202130,AEC
3,./2023-06-06/i=HAW&t=202130&s=AG,HAW,202130,AG
4,./2023-06-06/i=HAW&t=202130&s=AJ,HAW,202130,AJ
...,...,...,...,...
9289,./2023-06-06/i=WOA&t=202410&s=SP,WOA,202410,SP
9290,./2023-06-06/i=WOA&t=202410&s=SPED,WOA,202410,SPED
9291,./2023-06-06/i=WOA&t=202410&s=SSCI,WOA,202410,SSCI
9292,./2023-06-17/i=MAN&t=202340&s=JPN.html,MAN,202340,JPN


In [5]:
info_dicts = []
for i, path in enumerate(paths):
    with open(path, 'r') as f:
        text = f.read()
    html = bs(text, 'lxml')
    info_dicts.append({
        'path': path,
        'institute': html.title.text.split(' - ')[0],
        'term': html.title.text.split(' - ')[1],
        'school': html.h1.text.split(' (')[0],
        'table': html.table is not None,
        'updated': (html
            .find('p', attrs={'class': 'updated'}).text
            .strip('Updated: '))})
    print(f'\r{i + 1:<10}', end='')
print()

9294      


In [6]:
df_info = pd.DataFrame(info_dicts)
df_info['updated'] = (pd.to_datetime(
    df_info['updated'], 
    format='%m/%d/%Y %I:%M:%S %p %Z')
    .dt.date)

display(df_info)

,path,institute,term,school,table,updated
0,./2023-06-06/i=HAW&t=202130&s=ABRP,Hawaii Community College,Spring 2021,Auto Body Repair & Painting,True,2023-06-06
1,./2023-06-06/i=HAW&t=202130&s=ACC,Hawaii Community College,Spring 2021,Accounting,True,2023-06-06
2,./2023-06-06/i=HAW&t=202130&s=AEC,Hawaii Community College,Spring 2021,Architl Engineerg & CAD Tech,True,2023-06-06
3,./2023-06-06/i=HAW&t=202130&s=AG,Hawaii Community College,Spring 2021,Agriculture,True,2023-06-06
4,./2023-06-06/i=HAW&t=202130&s=AJ,Hawaii Community College,Spring 2021,Administration of Justice,True,2023-06-06
...,...,...,...,...,...,...
9289,./2023-06-06/i=WOA&t=202410&s=SP,University of Hawaii West Oahu,Fall 2023,Speech,True,2023-06-06
9290,./2023-06-06/i=WOA&t=202410&s=SPED,University of Hawaii West Oahu,Fall 2023,Special Education,True,2023-06-06
9291,./2023-06-06/i=WOA&t=202410&s=SSCI,University of Hawaii West Oahu,Fall 2023,Social Sciences,True,2023-06-06
9292,./2023-06-17/i=MAN&t=202340&s=JPN.html,University of Hawaii at Manoa,Summer 2023,Japanese Language & Literature,True,2023-06-17


In [7]:
df_merge = df_query.merge(df_info, how='left')

df_institute = (
    df_merge[[
        'institute_id',
        'institute']]
    .drop_duplicates()
    .sort_values(by='institute_id', ignore_index=True)
    .set_index('institute_id', verify_integrity=True)
    .reset_index())

df_term_temp = (
    df_merge[[
        'term_id',
        'term']]
    .drop_duplicates()
    .sort_values(by='term_id', ignore_index=True)
    .set_index('term_id', verify_integrity=True)
    .reset_index())

df_school = (
    df_merge[[
        'school_id',
        'school']]
    .drop_duplicates()
    .sort_values(by='school_id', ignore_index=True)
    .set_index('school_id', verify_integrity=True)
    .reset_index())

df_page = (
    df_merge[[
        'path',
        'institute_id',
        'term_id',
        'school_id',
        'table',
        'updated',]]
    .drop_duplicates()
    .sort_values(by='path', ignore_index=True)
    .set_index('path', verify_integrity=True)
    .reset_index())

display(df_institute)
display(df_term_temp)
display(df_school)
display(df_page)

,institute_id,institute
0,HAW,Hawaii Community College
1,HIL,University of Hawaii at Hilo
2,HON,Honolulu Community College
3,KAP,Kapi'olani Community College
4,KAU,Kauai Community College
5,LEE,Leeward Community College
6,MAN,University of Hawaii at Manoa
7,MAU,University of Hawaii Maui College
8,WIN,Windward Community College
9,WOA,University of Hawaii West Oahu


,term_id,term
0,200340,Summer 2003
1,200410,Fall 2003
2,200430,Spring 2004
3,200440,Summer 2004
4,200510,Fall 2004
...,...,...
60,202333,Spring 2023 Extension
61,202340,Summer 2023
62,202410,Fall 2023
63,202413,Fall 2023 Extension


,school_id,school
0,ABRP,Auto Body Repair & Painting
1,ACC,Accounting
2,ACM,Academy for Creative Media
3,AEC,Architl Engineerg & CAD Tech
4,AERO,Aeronautics
...,...,...
301,VIET,Vietnamese
302,WELD,Welding
303,WGSS,"Women,Gender&Sexuality Studies"
304,WS,Women's Studies


,path,institute_id,term_id,school_id,table,updated
0,./2023-06-06/i=HAW&t=202130&s=ABRP,HAW,202130,ABRP,True,2023-06-06
1,./2023-06-06/i=HAW&t=202130&s=ACC,HAW,202130,ACC,True,2023-06-06
2,./2023-06-06/i=HAW&t=202130&s=AEC,HAW,202130,AEC,True,2023-06-06
3,./2023-06-06/i=HAW&t=202130&s=AG,HAW,202130,AG,True,2023-06-06
4,./2023-06-06/i=HAW&t=202130&s=AJ,HAW,202130,AJ,True,2023-06-06
...,...,...,...,...,...,...
9289,./2023-06-06/i=WOA&t=202410&s=SP,WOA,202410,SP,True,2023-06-06
9290,./2023-06-06/i=WOA&t=202410&s=SPED,WOA,202410,SPED,True,2023-06-06
9291,./2023-06-06/i=WOA&t=202410&s=SSCI,WOA,202410,SSCI,True,2023-06-06
9292,./2023-06-17/i=MAN&t=202340&s=JPN.html,MAN,202340,JPN,True,2023-06-17


In [8]:
season_dict = {
    'Spring': 'March 1',
    'Summer': 'June 1',
    'Fall': 'September 1',
    'Winter': 'December 1'}

df_term = df_term_temp.copy()
df_term['academic'] = df_term['term_id'].astype('string').str[:4]
df_term['extension'] = df_term['term'].str.contains('Extension')
df_term['term'] = df_term['term'].str.strip(' Extension')
df_term[['semester', 'year']] = df_term['term'].str.split(' ', expand=True)
df_term['start'] = df_term['semester'].replace(season_dict) + ' ' + df_term['year']
df_term['start'] = pd.to_datetime(df_term['start'], format='%B %d %Y').dt.date
df_term = df_term.astype({'academic': 'int64', 'year': 'int64'})
df_term = df_term[[
    'term_id',
    'academic',
    'year',
    'semester',
    'extension',
    'start']]

display(df_term)

,term_id,academic,year,semester,extension,start
0,200340,2003,2003,Summer,False,2003-06-01
1,200410,2004,2003,Fall,False,2003-09-01
2,200430,2004,2004,Spring,False,2004-03-01
3,200440,2004,2004,Summer,False,2004-06-01
4,200510,2005,2004,Fall,False,2004-09-01
...,...,...,...,...,...,...
60,202333,2023,2023,Spring,True,2023-03-01
61,202340,2023,2023,Summer,False,2023-06-01
62,202410,2024,2023,Fall,False,2023-09-01
63,202413,2024,2023,Fall,True,2023-09-01


In [9]:
with open('df_institute', 'wb') as f:
    pickle.dump(df_institute, f)
    
with open('df_term', 'wb') as f:
    pickle.dump(df_term, f)
    
with open('df_school', 'wb') as f:
    pickle.dump(df_school, f)
    
with open('df_page', 'wb') as f:
    pickle.dump(df_page, f)